# explore gold and silver using FRED data

In [ ]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)
options("scipen"=999)

source("goofy/misc.common.R")
source("goofy/plot.common.R")
source("config.R")

fred<-Fred()
indices<-Indices()

first, lets check what kind of data is available on the FRED for gold and silver

In [ ]:
goldFred <- fred$Meta() %>% inner_join(fred$TimeSeries(), by = 'SERIES_ID') %>%
    filter(NAME %like% '%gold%') %>%
    group_by(SERIES_ID, TICKER, NAME) %>%
    summarize(MIN_TS = min(TIME_STAMP), MAX_TS = max(TIME_STAMP)) %>%
    select(SERIES_ID, TICKER, NAME, MIN_TS, MAX_TS)

print(data.frame(goldFred))

use the afternoon USD fix

In [ ]:
seriesIdGold <- -2147252004
goldDf <- data.frame(fred$TimeSeries() %>% filter(SERIES_ID == seriesIdGold, TIME_STAMP >= as.Date('1995-01-01')) %>% select(TIME_STAMP, VAL))
goldDf$TIME_STAMP <- as.Date(goldDf$TIME_STAMP)

fredDetailsGold <- data.frame(goldFred %>% filter(SERIES_ID == seriesIdGold))

print(nrow(goldDf))
print(head(goldDf))

In [ ]:
firstDate <- min(goldDf$TIME_STAMP)
lastDate <- max(goldDf$TIME_STAMP)
xAxisTicks <- seq(from=firstDate, to=lastDate, length.out=10)



options(repr.plot.width=16, repr.plot.height=8)

ggplot(goldDf, aes(x=TIME_STAMP, y=VAL)) +
    theme_economist() +
    #theme(text = element_text(size = 3), element_line(size = 0.1)) +
    geom_line() +
    scale_y_log10() +
    scale_x_date(breaks = xAxisTicks) +
    labs(x='', y='log(price)', color='', title=sprintf("FRED: %s - %s", fredDetailsGold$TICKER[1], fredDetailsGold$NAME[1]), 
         subtitle=sprintf("[%s:%s]", firstDate, lastDate)) +
    annotate("text", x=lastDate, y=min(goldDf$VAL, na.rm=T), label = "@StockViz", hjust=1.1, vjust=-1.1, col="white", cex=6, fontface = "bold", alpha = 0.8)

### NIFTY 50 vs. Gold

The NIFTY 50 TR index has been around since 1999. Translate that index to USD and plot monthly returns vs. gold.

In [ ]:
trIndex <- data.frame(indices$NseTimeSeries() %>%
    filter(NAME == "NIFTY 50 TR" && TIME_STAMP >= '2000-01-01' && TIME_STAMP <= lastDate) %>%
    select(TIME_STAMP, CLOSE))

trXts <- xts(trIndex$CLOSE, as.Date(trIndex$TIME_STAMP))

In [ ]:
seriesIdUsdInr <- -2147478748
usdInrDf <- data.frame(fred$TimeSeries() %>% filter(SERIES_ID == seriesIdUsdInr, TIME_STAMP >= as.Date('2000-01-01')) %>% select(TIME_STAMP, VAL))
usdInrDf$TIME_STAMP <- as.Date(usdInrDf$TIME_STAMP)

fredDetailsUsdInr <- data.frame(fred$Meta() %>% filter(SERIES_ID == seriesIdUsdInr))

print(nrow(usdInrDf))
print(head(usdInrDf))

usdInrXts <- xts(usdInrDf$VAL, usdInrDf$TIME_STAMP)

In [ ]:
allXts <- merge(trXts, usdInrXts)
names(allXts) <-c('TR', 'USDINR')

#holidays don't match up, so carry the USDINR forward.
allXts[,2] <- na.locf(allXts[,2])
allXts <- na.omit(allXts)

In [ ]:
#end-of months don't match, so force them to a common on
monthlies <- merge(Common.NormalizeMonthlyDates(monthlyReturn(allXts[,1])), Common.NormalizeMonthlyDates(monthlyReturn(allXts[,2])))